In [ ]:
!git clone https://github.com/brendenlake/omniglot.git
%cd omniglot/python
!unzip images_evaluation.zip
!unzip images_background.zip
%cd ../..

In [2]:
import zipfile
from pathlib import Path
import glob
import os, sys

import numpy as np
import random

In [3]:
CUR_DIR = Path.cwd()
CUR_DIR

PosixPath('/content')

In [4]:
DATA_PATH = CUR_DIR / 'omniglot/python'
DATA_PATH

PosixPath('/content/omniglot/python')

In [5]:
BACKGROUND = DATA_PATH / 'images_background'
EVALUATION = DATA_PATH / 'images_evaluation'
print(BACKGROUND)
print(EVALUATION)

/content/omniglot/python/images_background
/content/omniglot/python/images_evaluation


In [6]:
backgrounds = [x for x in glob.glob(os.path.join(BACKGROUND, '*'))]
evaluations = [x for x in glob.glob(os.path.join(EVALUATION, '*'))]
print(len(backgrounds))
print(len(evaluations))

30
20


In [7]:
pick_10 = list(np.random.choice(evaluations, size=10, replace=False))
print(len(pick_10))

backgrounds += pick_10
print(len(backgrounds))

evaluations = [x for x in evaluations if x not in pick_10]
print(len(evaluations))

10
40
10


In [8]:
SEED = 42

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as dset
from torchvision import transforms
from PIL import Image

In [10]:
class TrainDataset(Dataset):
    def __init__(self, path_list, transform=None):
        super(TrainDataset, self).__init__()
        np.random.seed(SEED)
    
        self.transform = transform
        self.data, self.n_classes, self.len = self.load_data(path_list)
        
    def load_data(self, path_list):
        print('Start loading the data')
        data = {}
        idx, elements = 0, 0
        for path in path_list:
            #print(path)
            for char_path in os.listdir(path):
                if isinstance(char_path, bytes):
                    char_path = char_path.decode('utf-8')
                #print(char_path, type(char_path))
                data[idx] = []
                for img_path in os.listdir(os.path.join(path, char_path)):
                    img = os.path.join(path, char_path, img_path)
                    data[idx].append(Image.open(img).convert('L'))
                elements += len(data[idx])
                idx += 1
        print('Complete the loading')
        return data, idx, elements
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        label, img1, img2 = None, None, None
        
        # get two images from "same" class
        if idx % 2 == 1:
            label = 1 # same
            idx1 = random.randint(0, self.n_classes - 1)
            img1 = random.choice(self.data[idx1])
            img2 = random.choice(self.data[idx1])
    
        # get two images from "different" class
        else:
            label = 0 # different
            idx1 = random.randint(0, self.n_classes - 1)
            idx2 = random.randint(0, self.n_classes - 1)
            
            # idx1 and idx2 must be different
            while idx1 == idx2:
                idx2 = random.randint(0, self.n_classes - 1)
            img1 = random.choice(self.data[idx1])
            img2 = random.choice(self.data[idx2])
        
        # transform the data if exists
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2, torch.from_numpy(np.array([label], dtype=np.float32))
            

In [34]:
class TestDataset(Dataset):
    def __init__(self, path_list, transform=None, times=200, way=20):
        super(TestDataset, self).__init__()
        np.random.seed(SEED + 1)
    
        self.transform = transform
        self.times = times
        self.way = way
        self.img1 = None
        self.c1 = None
        self.data, self.n_classes, self.len = self.load_data(path_list)
        
    def load_data(self, path_list):
        print('Start loading the data')
        data = {}
        idx, elements = 0, 0
        for path in path_list:
            for char_path in os.listdir(path):
                data[idx] = []
                for img_path in os.listdir(os.path.join(path, char_path)):
                    img = os.path.join(path, char_path, img_path)
                    data[idx].append(Image.open(img).convert('L'))
                elements += len(data[idx])
                idx += 1
        print('Complete the loading')
        return data, idx, elements
    
    def __len__(self):
        return self.len
        #return self.times * self.way
    
    def __getitem__(self, idx):
        i = idx % self.way
        label = None
        if i == 0:
            self.c1 = random.randint(0, self.n_classes - 1)
            self.img1 = random.choice(self.data[self.c1])
            img2 = random.choice(self.data[self.c1])
            label = 1
        else:
            c2 = random.randint(0, self.n_classes - 1)
            while self.c1 == c2:
                c2 = random.randint(0, self.n_classes - 1)
            img2 = random.choice(self.data[c2])
            label = 0
            
        if self.transform:
            img1 = self.transform(self.img1)
            img2 = self.transform(img2)
            
        return img1, img2, torch.from_numpy(np.array([label], dtype=np.float32))
            

In [35]:
transform_dataset = transforms.Compose([
    transforms.RandomAffine(10),
    transforms.ToTensor(),
])

train_set = TrainDataset(backgrounds, transform=transform_dataset)
test_set = TestDataset(evaluations, transform=transforms.ToTensor())

Start loading the data
Complete the loading
Start loading the data
Complete the loading


In [36]:
BATCH_SIZE = 32
TIMES = 400
WAY = 20

train_loader = DataLoader(train_set, 
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          )
test_loader = DataLoader(test_set,
                         batch_size=WAY,
                         shuffle=False,
                        )

In [37]:
import torch.nn as nn
import torch.nn.functional as F

class SiameseNet(nn.Module):
    def __init__(self):
        super(SiameseNet, self).__init__()
        self.conv = nn.Sequential(
                nn.Conv2d(1, 64, 10),  # 64@96*96
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),  # 64@48*48
                
                nn.Conv2d(64, 128, 7),
                nn.ReLU(),    # 128@42*42
                nn.MaxPool2d(2),   # 128@21*21
                
                nn.Conv2d(128, 128, 4),
                nn.ReLU(), # 128@18*18
                nn.MaxPool2d(2), # 128@9*9
                
                nn.Conv2d(128, 256, 4),
                nn.ReLU(),   # 256@6*6
            )
        self.linear = nn.Sequential(nn.Linear(9216, 4096), nn.Sigmoid())
        self.out = nn.Linear(4096, 1)
        
    def forward_1(self, x):
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        x = self.linear(x)
        return x
    
    def forward(self, x1, x2):
        out1 = self.forward_1(x1)
        out2 = self.forward_1(x2)
        L1_dist = torch.abs(out1 - out2)
        out = self.out(L1_dist)
        return out

In [38]:
is_cuda = torch.cuda.is_available()
is_cuda

True

In [39]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import time
from fastprogress import progress_bar
import datetime


model = SiameseNet()
if is_cuda:
    model.cuda()

criterion = nn.BCEWithLogitsLoss(size_average=True)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
#                                                        'min', 
#                                                         patience=10)

EPOCHS = 100
SHOW_EVERY = 50
TEST_EVERY = 50
LR = 0.001

train_loss = []
loss_val = 0

time_start = time.time()
for epoch in progress_bar(range(1, EPOCHS + 1)):
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                       'min', 
                                                        patience=10)    
    train_loss = []
    time_start = time.time()
    for batch, (img1, img2, label) in enumerate(progress_bar(train_loader), 1):
        #print(batch)
        model.train()
        optimizer.zero_grad()
        if is_cuda:
            img1, img2, label = img1.cuda(), img2.cuda(), label.cuda()
        output = model.forward(img1, img2)
        loss = criterion(output, label)
        train_loss.append(loss.item())
        loss_val += loss.item()
        
        loss.backward()
        optimizer.step()
        
        if batch % SHOW_EVERY == 0:
            print(f'[{epoch}][{batch}] Loss: {loss_val / SHOW_EVERY:.4f}, time_lapsed: {time.time() - time_start:.4f}')
            loss_val = 0
            time_start = time.time()
        
    # per-epoch
    model.eval()
    test_loss = 0
    correct, wrong = 0, 0
    
    with torch.no_grad():
        for (img1, img2, label) in progress_bar(test_loader):
            if is_cuda:
                img1, img2, label = img1.cuda(), img2.cuda(), label.cuda()
            output = model(img1, img2) #.data.cpu().numpy()
            test_loss += criterion(output, label)
            output = output.data.cpu().numpy()
            pred = np.argmax(output)
            if pred == 0:
                correct += 1
            else:
                wrong += 1
    
    print('*' * 50)
    acc = correct / (correct + wrong)
    test_loss /= len(test_loader)
    print(f'[{epoch}]| Test Loss: {test_loss:.4f}, correct: {correct}, wrong: {wrong}, accuracy: {acc:.4f}')
    print('*' * 50)
    #scheduler.step(test_loss)
    LR = LR * 0.99


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


[1][50] Loss: 0.6413, time_lapsed: 9.5927
[1][100] Loss: 0.6308, time_lapsed: 9.5114
[1][150] Loss: 0.5881, time_lapsed: 9.5144
[1][200] Loss: 0.5633, time_lapsed: 9.5190
[1][250] Loss: 0.5648, time_lapsed: 9.5346
[1][300] Loss: 0.5424, time_lapsed: 9.5375
[1][350] Loss: 0.5516, time_lapsed: 9.5476
[1][400] Loss: 0.4967, time_lapsed: 9.5421
[1][450] Loss: 0.5274, time_lapsed: 9.5410
[1][500] Loss: 0.5034, time_lapsed: 9.5482
[1][550] Loss: 0.4891, time_lapsed: 9.5481
[1][600] Loss: 0.4687, time_lapsed: 9.5552
[1][650] Loss: 0.4647, time_lapsed: 9.5615
[1][700] Loss: 0.4574, time_lapsed: 9.5756
[1][750] Loss: 0.4571, time_lapsed: 9.5715
[1][800] Loss: 0.4548, time_lapsed: 9.5646


**************************************************
[1]| Test Loss: 0.5281, correct: 98, wrong: 235, accuracy: 0.2943
**************************************************


[2][50] Loss: 0.4801, time_lapsed: 9.4519
[2][100] Loss: 0.3994, time_lapsed: 9.5476
[2][150] Loss: 0.4243, time_lapsed: 9.5427
[2][200] Loss: 0.4211, time_lapsed: 9.5464
[2][250] Loss: 0.3986, time_lapsed: 9.5541
[2][300] Loss: 0.3835, time_lapsed: 9.5476
[2][350] Loss: 0.3859, time_lapsed: 9.5432
[2][400] Loss: 0.3623, time_lapsed: 9.5451
[2][450] Loss: 0.3636, time_lapsed: 9.5464
[2][500] Loss: 0.3562, time_lapsed: 9.5465
[2][550] Loss: 0.3533, time_lapsed: 9.5397
[2][600] Loss: 0.3524, time_lapsed: 9.5372
[2][650] Loss: 0.3539, time_lapsed: 9.5474
[2][700] Loss: 0.3345, time_lapsed: 9.5504
[2][750] Loss: 0.3190, time_lapsed: 9.5544
[2][800] Loss: 0.3014, time_lapsed: 9.5446


**************************************************
[2]| Test Loss: 0.4360, correct: 165, wrong: 168, accuracy: 0.4955
**************************************************


...

[56][50] Loss: 0.0690, time_lapsed: 9.4370
[56][100] Loss: 0.0762, time_lapsed: 9.5236
[56][150] Loss: 0.0637, time_lapsed: 9.5235
[56][200] Loss: 0.0595, time_lapsed: 9.5149
[56][250] Loss: 0.0554, time_lapsed: 9.5203
[56][300] Loss: 0.0640, time_lapsed: 9.5171
[56][350] Loss: 0.0674, time_lapsed: 9.5231
[56][400] Loss: 0.0635, time_lapsed: 9.5269
[56][450] Loss: 0.0661, time_lapsed: 9.5262
[56][500] Loss: 0.0559, time_lapsed: 9.5209
[56][550] Loss: 0.0602, time_lapsed: 9.5254
[56][600] Loss: 0.0563, time_lapsed: 9.5140
[56][650] Loss: 0.0806, time_lapsed: 9.5219
[56][700] Loss: 0.0548, time_lapsed: 9.5304
[56][750] Loss: 0.0651, time_lapsed: 9.5228
[56][800] Loss: 0.0548, time_lapsed: 9.5274


**************************************************
[56]| Test Loss: 0.1228, correct: 271, wrong: 62, accuracy: 0.8138
**************************************************


[57][50] Loss: 0.0890, time_lapsed: 9.4290
[57][100] Loss: 0.0556, time_lapsed: 9.5172
[57][150] Loss: 0.0542, time_lapsed: 9.5222
[57][200] Loss: 0.0681, time_lapsed: 9.5199
[57][250] Loss: 0.0623, time_lapsed: 9.5275
[57][300] Loss: 0.0607, time_lapsed: 9.5238
[57][350] Loss: 0.0690, time_lapsed: 9.5226
[57][400] Loss: 0.0687, time_lapsed: 9.5228
[57][450] Loss: 0.0774, time_lapsed: 9.5244
[57][500] Loss: 0.0614, time_lapsed: 9.5168
[57][550] Loss: 0.0530, time_lapsed: 9.5205
[57][600] Loss: 0.0561, time_lapsed: 9.5237
[57][650] Loss: 0.0618, time_lapsed: 9.5251
[57][700] Loss: 0.0633, time_lapsed: 9.5170
[57][750] Loss: 0.0580, time_lapsed: 9.5209
[57][800] Loss: 0.0963, time_lapsed: 9.5304


**************************************************
[57]| Test Loss: 0.1080, correct: 273, wrong: 60, accuracy: 0.8198
**************************************************


[58][50] Loss: 0.0802, time_lapsed: 9.4345
[58][100] Loss: 0.0573, time_lapsed: 9.5217
[58][150] Loss: 0.0626, time_lapsed: 9.5228
[58][200] Loss: 0.0639, time_lapsed: 9.5163
[58][250] Loss: 0.0462, time_lapsed: 9.5189
[58][300] Loss: 0.0719, time_lapsed: 9.5180
[58][350] Loss: 0.0583, time_lapsed: 9.5191
[58][400] Loss: 0.0537, time_lapsed: 9.5234
[58][450] Loss: 0.0771, time_lapsed: 9.5253
[58][500] Loss: 0.0643, time_lapsed: 9.5148
[58][550] Loss: 0.0525, time_lapsed: 9.5213
[58][600] Loss: 0.0780, time_lapsed: 9.5247


In [ ]:
SAVE_PATH = CUR_DIR / 'sample_data' / 'siamese.pt'
torch.save(model, SAVE_PATH)